<a href="https://colab.research.google.com/github/Serge3leo/temp-cola/blob/main/stackoverflow-8585880.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required packages
!apt-get update -qq
!apt-get install -qq gfortran
!pip install -qq fortran-magic
# Workaround colab bug
!pip install -qq meson charset-normalizer ninja cmake pkgconfig

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
%load_ext fortranmagic
# Workaround colab bug
%fortran_config --backend meson -v

The fortranmagic extension is already loaded. To reload it, use:
  %reload_ext fortranmagic
New default arguments for %fortran:
	--backend meson -v


In [4]:
%%fortran -v
subroutine read_params(filename, params, ierr)
   implicit none
   ! Argument Declarations !
   character(len=*), intent(in) :: filename
   integer, dimension(4), intent(out) :: params
   integer, intent(out) :: ierr
   ! Variable Declarations
   integer :: i, iounit

   iounit = 1
   ierr = -2 ! Initialize error code for open error
   open(unit=iounit, status="old", file=filename, form="unformatted", err=10)
   ierr = -1 ! Initialize error code for read error
   read(iounit, *, err=10) (params(i), i=1, 4)
   ierr = 0 ! OK
10 close(iounit)

end subroutine read_params


Ok. The following fortran objects are ready to use: read_params


In [5]:
params, ierr = read_params("not-file")
ierr

-2

In [6]:
%%fortran -v
subroutine read_params_msg(filename, params, ierr, errmsg)
   implicit none
   ! Argument Declarations !
   character(len=*), intent(in) :: filename
   integer, dimension(4), intent(out) :: params
   integer, intent(out) :: ierr
   character(len=256), intent(out) :: errmsg
   ! Variable Declarations
   integer :: i, iounit

   iounit = 1
   errmsg = ""
   ierr = -2 ! Initialize error code for open error
   open(unit=iounit, status="old", file=filename, form="unformatted", &
        err=10, iomsg=errmsg)
   ierr = -1 ! Initialize error code for read error
   read(iounit, *, err=10, iomsg=errmsg) (params(i), i=1, 4)
   ierr = 0 ! OK
10 close(iounit)

end subroutine read_params_msg


Ok. The following fortran objects are ready to use: read_params_msg


In [7]:
params, ierr, errmsg = read_params_msg("not-file")
ierr, errmsg

(-2, b"Cannot open file 'not-file': No such file or directory")